In [ ]:
!df -h

## Library

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

## Load Config

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'cascade_mask_rcnn_swin_base_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco'
folder_name = 'swin'

# config file 들고오기
cfg = Config.fromfile(f'./configs/{folder_name}/{model_name}.py')

## Hyper-parameters

In [ ]:
# batch size
cfg.data.samples_per_gpu = 3
cfg.data.workers_per_gpu = 4

# epochs
# cfg.runner.max_epochs = 25 

# learning rate
# cfg.optimizer.lr = 1e-4

# scheduler
# cfg.lr_config = dict(
#     policy='CosineAnnealing',
#     warmup='linear',
#     warmup_iters=1000,
#     warmup_ratio=1.0 / 10,
#     min_lr_ratio=1e-6
# )

## Config Check

In [ ]:
import pprint
pprint.pprint(cfg.lr_config)

In [ ]:
pprint.pprint(cfg.optimizer)

In [ ]:
pprint.pprint(cfg.data)

In [ ]:
pprint.pprint(cfg.model)

## Config Tuning

In [ ]:
PREFIX = '../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
# cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.ann_file = PREFIX + 'train_all.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


cfg.seed=0
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{model_name}'

# cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

## Train

In [ ]:
model = build_detector(cfg.model)

In [ ]:
datasets = [build_dataset(cfg.data.train)]

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

## Save Config

In [ ]:
import os
from mmcv import Config

# ./configs/{model_name}/{model_cfg}
# Example)
# ./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py

# config 불러오기.
cfg = Config.fromfile(os.path.join("./configs", folder_name, f'{model_name}.py'))

# config 출력하기.
# 모든 설정 값이 출력됩니다요.
print(cfg.dump())

# 현재 경로의 _custom_configs_ 폴더를 만들고 거기다가 config 파일을 저장합니다.
os.makedirs(os.path.join("_custom_configs_"), exist_ok=True)
cfg.dump(os.path.join("_custom_configs_", model_name))